In [7]:
import vonkarman as vk
import numpy as np

In [8]:
"""This is the code I used to save the npz file"""

# import cfGPR
# exposure = 465
# GP = cfGPR.CurlFreeGPR()
# GP.load_fits("folio2")
# GP.extract_data(exposure)
# GP.sigma_clip()
# GP.split_data(train_size=0.50)

# X = GP.X # Star positions (deg)
# Y = GP.Y # Astrometric residuals (mas)

# np.savez("stars.npz", X=X, Y=Y)

'This is the code I used to save the npz file'

In [9]:
def getGrid(X1, X2):
    u1, u2 = X1[:, 0], X2[:, 0]
    v1, v2 = X1[:, 1], X2[:, 1]
    uu1, uu2 = np.meshgrid(u1, u2)
    vv1, vv2 = np.meshgrid(v1, v2)
    
    return uu1 - uu2, vv1 - vv2

In [10]:
stars = np.load("stars.npz")
X = stars['X']
Y = stars['Y']
X.shape, Y.shape

((8138, 2), (8138, 2))

In [11]:
# For this exposure, this is what the optimizer found the parameters to be
p0 = np.array([
    409.29434,
    4.19736,
    0.05170,
    -0.04572,
    -0.08203
])

ttt = vk.TurbulentLayer(
    variance=p0[0],
    outerScale=p0[1],
    diameter=p0[2],
    wind=(p0[3], p0[4]),
    dWind = p0[2]/5.)

In [12]:
#xx,yy = getGrid(X[::2,:],X[::2,:])
xx,yy = getGrid(X,X)

In [13]:
%%time
cov = ttt.getCuv(xx,yy)

/home/fortino/DESworkspace/vonkarman.py:187: RuntimeWarning: divide by zero encountered in log
  return  self.interpolator(np.log(r))


CPU times: user 58min 31s, sys: 4min 35s, total: 1h 3min 6s
Wall time: 5min 23s


In [14]:
n = cov.shape[0]
c22 = np.swapaxes(cov,1,2).reshape(2*n,2*n)

In [15]:
e=np.linalg.eigvalsh(c22+np.eye(c22.shape[0]))
print(e)

[9.99697212e-01 9.99791716e-01 9.99856780e-01 ... 9.22249097e+04
 9.28224249e+04 1.13894148e+05]


In [16]:
L = np.linalg.cholesky(c22+np.eye(c22.shape[0]))